# Auction algorithm for the CSP

## Problem

Let $(G, E)$ denote a directed graph. To each arc $(i, j) \in E$ we associate a cost $c_{ij}$ and a length $l_{ij}$. Let $s$ and $t$ be two distinct nodes in $G$. We consider the Constrained Shortest Path problem formulated as a mixed integer program. $A x = d$ denotes the usual network flow constraint for the shortest path:

$$
\begin{alignat*}{3}
& \text{CSP} \quad=\quad
    && \text{minimize}   \quad && c^T x \\
&   && \text{subject to} \quad && A x = d \\
&   &&                         && l^T x \leq R \\
&   &&                         && x \in \lbrace 0, 1 \rbrace^|E| \\
\end{alignat*}
$$

## Auction algorithm for the shortest path

Bertsekas introduced an [auction algorithm for the shortest path](https://web.mit.edu/dimitrib/www/Auctionsp.pdf) in 2001. The idea is to maintain a pair $(P, \pi)$ where $P$ is a path starting at $s$, and $\pi$ is a vector over $G$ such that:

$$
\begin{align*}
& \pi_i - \pi_j \leq c_{ij} \\
& \pi_i - \pi_j = c_{ij} \quad \forall ij \in P \\
\end{align*}
$$

These are easily understood as the complementary slackness conditions for the shortest path LP formulation. The path $P$ is therefore always a shortest path between its end nodes. $P$ is successively contracted and extended with a simple rule. The algorithm terminates when it reaches $t$.

## An auction algorithm for the CSP

We can naturally add a constraint on $P$ so that its length never exceeds $R$, thus garanteeing that if the algorithm terminates, the shortest path found is feasible for the CSP. Whenever the auction algorithm chooses which node becomes a candidate to extend the path, we exclude those nodes which would create an infeasible path.

This modified auction algorithm simulates the auction algorithm running on an auxiliary graph, which we describe here:
- Denote the auxiliary graph $(G', E')$,
- The length and cost of an arc are respectively denoted $l_{ij}$ and $c_{ij}$.
- Let $G' = \lbrace (i, l) : i \in G,\: l \leq R \text{ and there exists an elementary path from } s \text{ to } i \text{ of length } l \rbrace$
- There is an arc in $E'$ from $(i, l_i)$ to $(j, l_j)$ in $G'$ if $l_{ij} = l_j - l_i$. This arc has cost $c_{ij}$.

If the CSP is feasible, the modified auction algorithm terminates as soon as it reaches a node in $T = \lbrace (i, l) \in G' : i = t \rbrace$. If the CSP is infeasible, the auction algorithm never terminates. A way to get around this is to provide an upper bound $M$ to the CSP value and virtually connect the source node to the destionation with length $0$ and cost $M$. The more accurate the upper bound, the faster the algorithm terminates.

In [150]:
from dataclasses import dataclass
import heapq
import math


@dataclass(slots=True)
class Arc:
    l: float
    c: float


Node = int
Adj = dict[Node, dict[Node, Arc]]


def csp(adj: Adj, s: Node, t: Node, R: float = math.inf, M: float = math.inf):
    path = [s]
    elm = {s}
    p = {i: 0.0 for i in adj}
    l = 0

    it = 0
    while path[-1] != t:
        i = path[-1]

        j, pij = t, M
        for _j, e in adj[i].items():
            if (_pij := e.c + p[_j]) < pij and (l + e.l <= R) and (_j not in elm):
                j, pij = _j, _pij

        if p[i] < pij:
            p[i] = pij

            if i != s:
                elm.remove(path.pop())
                l -= adj[path[-1]][i].l

        elif j in adj[i]:
            path.append(j)
            elm.add(j)
            l += adj[i][j].l

        else:
            path = None
            break

        it += 1

    return path

## Testing the algorithm on a random grid graph

In [151]:
import random
import itertools
import collections


n = 30

nodes = range(n * n)

edges = set()
for k in nodes:
    i, j = divmod(k, n)

    for ni, nj in [(i, j + 1), (i, j - 1), (i + 1, j), (i - 1, j)]:
        if (n * ni + nj) in nodes and 0 <= nj < n:
            edges.add((k, (n * ni + nj)))
            edges.add(((n * ni + nj), k))


adj = collections.defaultdict(dict)

for (i, j) in edges:
    adj[i][j] = Arc(1, random.randint(1, 5))

---

We find a path whose cost changes with the length constraint by brute force.

In [152]:
path = None

while path is None or path == csp(adj, src, dst):
    src = dst = random.randint(0, n * n - 1)
    while dst == src:
        dst = random.randint(0, n * n - 1)

    path = csp(adj, src, dst, 30, 5 * (n + n + 1))

In [153]:
def cost(adj, path):
    c = 0
    for i, j in zip(path, path[1:]):
        c += adj[i][j].c
    return c

spath = csp(adj, src, dst)
len(spath) - 1, len(path) - 1, cost(adj, spath), cost(adj, path)

(31, 27, 50, 51)

The constrained shortest path finds a feasible path of slightly superior cost than the unconstrained shortest path.

## A terrible benchmark

In [154]:
%%timeit

src = dst = random.randint(0, n * n - 1)
while dst == src:
    dst = random.randint(0, n * n - 1)

path = csp(adj, src, dst, 30, 5 * (n + n + 1))

The slowest run took 7.21 times longer than the fastest. This could mean that an intermediate result is being cached.
54.7 ms ± 28.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
